# Machine Learning

## Classification continued - more sophisticated models

Last time we talked about both decision trees and KNN models for doing classification problems. We noted that they are both relatively simple models (deliberately introduced for this reason). This time we want to show some other regression models that stretch more towards the state-of-the-art.

First we need to import all of the modules we are going to use. Notice that we will also grab the ensemble and cluster sub-modules of Scikit Learn as the main new elements for this notebook.

In [ ]:
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.neighbors as nei
import sklearn.tree as tree
import sklearn.ensemble as ens
import sklearn.cluster as clus
import sklearn.preprocessing as prepr
import sklearn.metrics as metrics
import numpy as np

First let's return to the synthetic data set we used before. This time we can read in both train and test csv files, and split them up into X and y train/test splits using the same pattern of variable names we used in the last notebook.

We can also make a quick scatter plot to look at our training data again. 

In [ ]:
train_data = pd.read_csv('synth.tr.csv')
test_data = pd.read_csv('synth.te.csv')

X_train = train_data[['xs','ys']]
y_train = train_data['yc']
X_test = test_data[['xs','ys']]
y_test = test_data['yc']


sns.scatterplot(data=train_data, x='xs', y='ys', hue='yc')
plt.title('Scatter plot - coloured by class')
plt.show()

Next we could fit a number of our classifiers to the training data, we can compare a decision tree with a gradient boosted tree, and a random forest (so, the two types of ensemble models we discussed). Each time we are going to fit our model using the training data, apply it to predict classes for our test data, and use those predictions to compare to the real classes to produce some scores for our various methods.

Notice how each of the three methods is very similar to implement in code - all we really change is the setup of the classifier model in question:

In [ ]:
dt_class = tree.DecisionTreeClassifier()
dt_class.fit(X_train,y_train)
y_predict=dt_class.predict(X_test)

print('DT Balanced accuracy: ',skl.metrics.balanced_accuracy_score(y_test,y_predict))
print('DT F1 score: ',skl.metrics.f1_score(y_true=y_test,y_pred=y_predict))

gbt_class = ens.HistGradientBoostingClassifier(max_iter=100)
gbt_class.fit(X_train,y_train)
y_predict=gbt_class.predict(X_test)

print('GBT Balanced accuracy: ',skl.metrics.balanced_accuracy_score(y_test,y_predict))
print('GBT F1 score: ',skl.metrics.f1_score(y_true=y_test,y_pred=y_predict))

rf_class = ens.RandomForestClassifier(n_estimators=10)
rf_class.fit(X_train,y_train)
y_predict=rf_class.predict(X_test)

print('RF Balanced accuracy: ',skl.metrics.balanced_accuracy_score(y_test,y_predict))
print('RF F1 score: ',skl.metrics.f1_score(y_true=y_test,y_pred=y_predict))

If you run and re-run the above code cell you will notice that the random forest score changes every time. This is due to the random forest being made of decision trees trained on random subsets of our training data, and so that split is slightly different each time.

Overall though you should generally notice that both ensemble methods tend to do better than the decision tree, and usually the gradient boosted tree does slightly better than the random forest.

The last model we fitted was the random forest, so we can have a look at what the classification looks like by making a plot of our training data and the classification of our test data:

In [ ]:
sns.scatterplot(data=test_data, x="xs", y="ys", color='green')
plt.title("Scatter plot of test data")
plt.show()


sns.scatterplot(data=test_data, x="xs", y="ys", hue=y_predict)
sns.scatterplot(data=train_data, x="xs", y="ys", hue='yc',legend=False,marker='^')
plt.title("Scatter plot of test & train data")
plt.show()

If you look back to week 7 you can see the similar plot we did for the KNN method.

## Clustering

Next we want to try out the K-Means clustering method on some data. To do this we will use the Penguins dataset - this has various measurements of different species of penguin. The dataset is built into the seaborn module as an example dataset, so we can load it in from there. 

First let's load in the data and get rid of any missing values by using the `dropna()` function from pandas to remove them:

In [ ]:
penguins = sns.load_dataset('penguins')
penguins.dropna(inplace=True)
print(penguins.head())

We can't view the entirety of this feature space as it is too high dimensional - but we can plot two of the columns together. We can also colour the points according to a third column (species in this case) to see how the groupings of data points look in this slice of our feature space.

In [ ]:
sns.scatterplot(data=penguins, x="bill_length_mm", y="bill_depth_mm", hue='species')
plt.title("Scatter plot of penguin data")
plt.show()

We can see broadly the three species look a bit like three clusters in this view. This prompts a question then - could we cluster using all of the columns except for 'species' into three clusters to see if we can see the three species coming out in the clusters that are found?

Here is the code to make our clustering - we need to supply two parameters to the K-Means clustering, firstly K, but also `n_init` which details how many times the algorithm starts from a different random start point to make sure it converges (we can pretty much always leave that as `auto` to let the algorithm automatically select how to do this). When we fit the clusters we might like to use all the data columns apart from the `species` column - but notice that some of them (island, sex) are categorical and so we can't immediately include them as they aren't numerical and we can't define a distance between points with them as they are. So for now we leave out those two columns in the following:

In [ ]:
clusters = clus.KMeans(n_clusters=3,n_init="auto")
clus_pred = clusters.fit(penguins[["bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g"]])
print(clus_pred.labels_)
print(clus_pred.cluster_centers_)

We can get both the cluster labels and the cluster centres (the two main outputs from the method) from the fitted model. Both are kept in variables with a trailing underscore `labels_` and `cluster_centers_`. The trailing underscore is a signal that these are internal to the method and not entirely meant to be used as output (but we can be cheeky and grab them as these are things we might want to use). The main output is gained from the `predict()` method of the clustering just like for classification methods - that predicts the cluster of an additional point.

We can plot our data again and this time use the cluster labels to colour the data points:

In [ ]:
sns.scatterplot(data=penguins, x="bill_length_mm", y="bill_depth_mm", hue=clus_pred.labels_,palette="pastel")
plt.title("Scatter plot of penguin data")
plt.show() 

As we discussed in the lecture there are two ways to shift a categorical variable into a numerical value. The first way is to assign an integer to each class within our categorical variable (so the column island has three possible values Torgersen, Biscoe, and Dream so these get converted to 0,1,2 in our new column.

The code for how to do this is below - first we convert the column to be of 'categorical' type in pandas, and then make a new column with the category codes (the numbered version of the categories):

In [ ]:
penguins["island"] = penguins["island"].astype('category')
penguins['island_no'] = penguins["island"].cat.codes

penguins["sex"] = penguins["sex"].astype('category')
penguins['sex_no'] = penguins["sex"].cat.codes
print(penguins)

Once we have these new columns we can include them in our clustering calculation. Here are the same lines again to do this but now with the added columns with encoded categories passed into our `fit()` function:

In [ ]:
clusters = clus.KMeans(n_clusters=3,n_init="auto")
clus_pred = clusters.fit(penguins[["bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g","sex_no","island_no"]])
print(clus_pred.labels_)
print(clus_pred.cluster_centers_)

And again we can plot up our data with this new clustering providing the colouring according to which cluster each point belongs to:

In [ ]:
sns.scatterplot(data=penguins, x="bill_length_mm", y="bill_depth_mm", hue=clus_pred.labels_,palette="pastel")
plt.title("Scatter plot of penguin data")
plt.show()

One of the other issues we worried about for distance-based methods is that distances in different feature directions should be similar. So each feature should be re-scaled to make sure that they vary by about the same amount. 

Sklearn has a preprocessing sub-module which contains a function to normalise our data automatically. Here we repeat the above clustering, but this time preprocess our data using the normalisation function first:

In [ ]:
clusters = clus.KMeans(n_clusters=3,n_init="auto")
X_normalized = prepr.normalize(penguins[["bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g","sex_no","island_no"]], norm='l2')
clus_pred = clusters.fit(X_normalized)
print(clus_pred.labels_)
print(clus_pred.cluster_centers_)

In [ ]:
sns.scatterplot(data=penguins, x="bill_length_mm", y="bill_depth_mm", hue=clus_pred.labels_,palette="pastel")
plt.title("Scatter plot of penguin data")
plt.show()

That seems perhaps a little better, we are beginning to see a similar sort of cluster split to the first plot where we coloured our data by species. 

Lastly we discussed in the lecture one other method for encoding our categorical data into a numerical version. This second method is the one-hot method where we make a new column for each category option in our variable and put 1s in the appropriate columns. Here's how we do that in code the `get_dummies` function in pandas does most of the work for us:

In [ ]:
one_hot_s = pd.get_dummies(penguins['sex'])
penguins = penguins.join(one_hot_s)
one_hot_i = pd.get_dummies(penguins['island'])
penguins = penguins.join(one_hot_i)
print(penguins)

Next we can repeat our normalised clustering using these one-hot encoded columns instead of the previous versions. Here is the code for that:

In [ ]:
clusters = clus.KMeans(n_clusters=3,n_init="auto")
X_normalized = prepr.normalize(penguins[["bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g","Torgersen","Biscoe","Dream","Male","Female"]], norm='l2')
clus_pred = clusters.fit(X_normalized)
print(clus_pred.labels_)
print(clus_pred.cluster_centers_)

Again we can plot using the colouring of our clusters:

In [ ]:
sns.scatterplot(data=penguins, x="bill_length_mm", y="bill_depth_mm", hue=clus_pred.labels_,palette="pastel")
plt.title("Scatter plot of penguin data")
plt.show()

One thing to note is that displayed in this way the clusters don't look very round and compact - but we could make the plot using our normalised columns instead and we would recover that view of our data - here's how that looks:

In [ ]:
penguins['norm1'] = X_normalized[:,0]
penguins['norm2'] = X_normalized[:,1]

sns.scatterplot(data=penguins, x='norm1', y='norm2', hue=clus_pred.labels_,palette="pastel")
plt.title("Scatter plot of scaled penguin data")
plt.show()

Lastly we talked about the way in which we might score our clustering using the silhouette measure, or looking at the average distance from points to their centre in each cluster.

The silhouette score is available in the metrics sub-module of Sklearn:

In [ ]:
metrics.silhouette_score(penguins[["bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g","sex_no","island_no"]], clus_pred.labels_)

This gives us the overall silhouette score for a given clustering allowing us to compare between different attempts at clustering a given datset.

We also talked in the lecture about how we can select the value of K for our K-Means algorithm. 

Below we construct a loop over different values of K and repeat our clustering for each value, each time computing the two measures we have considered, silhouette and cumulative distance from points to centre in each cluster (note Sklearn uses cumulative distance rather than average - but it makes little difference in the way we use these).

Here is the code to construct these scores and make suitable plots:

In [ ]:
kvals = np.arange(2,10)
scores = []
distsq = []
for k in kvals:
    clusters = clus.KMeans(n_clusters=k,n_init="auto")
    clus_pred = clusters.fit(penguins[["bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g","sex_no","island_no"]])
    score = metrics.silhouette_score(penguins[["bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g","sex_no","island_no"]], clus_pred.labels_)
    print(score)
    scores.append(score)
    print(clus_pred.inertia_)
    distsq.append(clus_pred.inertia_)
    
fig = plt.figure(1)
ax = fig.add_subplot(111)
ax.plot(kvals,scores)
plt.xlim([0,10])
plt.ylim([0.4,0.8])
plt.show()

fig = plt.figure(1)
ax = fig.add_subplot(111)
ax.plot(kvals,distsq)
plt.show()

From these plots K=2 gives the best silhouette score and so would be selected, for the elbow plot there isn't a very clear elbow, but arguments could be made in favour of selecting K between 4 and 6 by inspection of the plot.


## Exercises

1) Examine the dataset in `titanic.csv`. It contains data about travellers on the Titanic and whether or not they survived. This can be constructed as a classification problem - try loading the data and splitting it into both training and test data sets - you should use 25% of the data for the test dataset, note the classes we want to classify are contained in the column 'survived'.

2) Using both a 3-NN classifier and a random forest to predict the survival (the 'survived' column) of each passenger from the other data columns. Which method performs best?

3) Load up the `mpg.csv` dataset and discuss how many clusters we should optimally cluster it into, make sure to normalise the data before clustering it (use all the columns).